# Energy Mix
**Group 4**  
Jianyi Chen, Jincheng Hong, Tianyu Su, Yicheng Huang

## 1. Problem Statement

Planet 441 is a planet with natural conditions similar to those of B.C. It relies on a combination of renewable and non-renewable energy sources to meet its power and energy needs. While hydroelectricity is a significant contributor, other energy sources including coal and natural gas, as well as emerging renewable energy technologies such as wind and solar, also play a role. However, due to a special government type, the government has a fixed budget for the cost of energy production. Our goal is to develop a mathematical optimization model that minimizes pollution while ensuring that energy needs are met efficiently.

## 2. Variables and Parameters

| Parameter               | Symbol        | Unit                |
|-------------------------|--------------|--------------------|
| LCOE Cost              | c            | \$/Mwh             |
| Pollution Cost         | p            | g/MWh CO2eq        |
| Fixed Cost             | f            | \$ per MW           |
| Production Limit       | l            | MW                 |
| Renewable Indicator    | r            | Binary (0 or 1)    |
| Consumption Rate       | consumption_rate | Ratio           |
| Total Energy Demand    | D            | MW                 |
| Total Budget          | B            | \$                  |
| Carbon Tax            | carbon_tax   | \$ per tonne CO2eq  |
| Renewable Energy Target | $\lambda$       | Ratio              |

## 3. Assumptions and Constraints

**Task:** State assumptions and constraints. 



### **Objective Function**
Minimize the total pollution cost including carbon tax:
$$
\min \sum_{i \in \text{energy sources}} \left( \frac{p_i}{10^6} \times \text{carbon\_tax} \times x_i \right)
$$

### **Constraints**
#### **1. Energy Demand Constraint**
Ensuring the total extracted energy meets demand:
$$
\sum_{i \in \text{energy sources}} y_i \geq D
$$

#### **2. Production Limits**
Each energy source must not exceed its production capacity:
$$
x_i \leq l_i, \quad \forall i \in \text{energy sources}
$$
The extracted energy depends on the consumption rate:
$$
y_i = x_i \times \text{consumption\_rate}_i, \quad \forall i \in \text{energy sources}
$$

#### **3. Budget Constraint**
The total energy production cost must not exceed the allocated budget:
$$
\sum_{i \in \text{energy sources}} c_i \times x_i \leq B
$$

#### **4. Renewable Energy Constraint**
Ensuring at least \( R_{\min} \) proportion of energy comes from renewable sources:
$$
\sum_{i \in \text{renewable sources}} y_i \geq R_{\min} \times \sum_{i \in \text{energy sources}} y_i
$$


## 4. Build Solutions

**Task:** Construct and run the linear regression model where `concrete_compressive_strength` is the output and `cement`,`water`, `course_aggregate`, and `fine_aggregate` are the inputs.

## 5. Simple Model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pulp as lp
from pulp import LpProblem, LpVariable, LpMinimize
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

In [2]:
parameters_df = pd.read_csv("energy_parameters.csv", index_col=0)
parameters_df

,LCOE Cost ($/MWh),Pollution Cost (g/kWh CO2eq),Fixed Cost ($ per MW),Production Limit (MW),Renewable Indicator,Consumption Rate
Energy Source,,,,,,
solar,86.5,37,1516000,905.264,1,0.90
wind,106.5,12,1389000,905.264,1,0.85
hydro,213.0,150,6428500,43000.000,1,0.95
coal,74.0,1000,4074000,0.000,0,0.70
natural_gas,54.0,430,1300000,452.632,0,0.80


In [3]:
energy_sources = parameters_df.index.tolist()

c = parameters_df["LCOE Cost ($/MWh)"].to_dict()
p = parameters_df["Pollution Cost (g/kWh CO2eq)"].to_dict()
f = parameters_df["Fixed Cost ($ per MW)"].to_dict()
l = parameters_df["Production Limit (MW)"].to_dict()
r = parameters_df["Renewable Indicator"].to_dict()
consumption_rate = parameters_df["Consumption Rate"].to_dict()

D = 20000 # Total energy demand (MW)
B = 9.36e9 # Total budget ($)
carbon_tax = 10  # Carbon tax ($ per tonne CO2eq)
R_min = 0.8  # Updated renewable energy target to 0.8

x = LpVariable.dicts("x", energy_sources, lowBound=0)  # Energy supply
y = LpVariable.dicts("y", energy_sources, lowBound=0)  # Energy extraction amount

model = LpProblem("Energy_Optimization", LpMinimize)

model += lpSum((p[i] / 1e6) * carbon_tax * x[i] for i in energy_sources)

# demand
model += lpSum(y[i] for i in energy_sources) >= D

# production limits
for i in energy_sources:
    model += x[i] <= l[i]
    model += y[i] == x[i] * consumption_rate[i]  # Extraction amount = Production * Consumption rate

# Constraint: budget
model += lpSum(c[i] * x[i] for i in energy_sources) <= B

# Constraint: at least R_min percent renewable energy
renewable_energy = lpSum(y[i] * r[i] for i in energy_sources)
total_energy = lpSum(y[i] for i in energy_sources)
model += renewable_energy >= R_min * total_energy

model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a2de2273c6524090af7055daf5ba8b23-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/a2de2273c6524090af7055daf5ba8b23-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 54 RHS
At line 68 BOUNDS
At line 69 ENDATA
Problem MODEL has 13 rows, 10 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-12) rows, 4 (-6) columns and 4 (-26) elements
0  Obj 28.505664 Primal inf 1946.413 (1)
1  Obj 29.521139
Optimal - objective value 29.521139
After Postsolve, objective 29.521139, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 29.52113936 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock secon

1

## 4.1 simple model

In [40]:
import pulp

# Define the problem
prob = pulp.LpProblem("EnergyMixOptimization", pulp.LpMinimize)

# Given data
sources = ["solar", "wind", "hydro", "coal", "natural_gas"]
lcoe_cost = {"solar": 86.5, "wind": 106.5, "hydro": 213.0, "coal": 74.0, "natural_gas": 54.0}
pollution_cost = {"solar": 37, "wind": 12, "hydro": 150, "coal": 1000, "natural_gas": 430}
fixed_cost = {"solar": 1516000, "wind": 1389000, "hydro": 6428500, "coal": 4074000, "natural_gas": 1300000}
production_limit = {"solar": 999, "wind": 999, "hydro": 43000.0*1000, "coal": 200*1000, "natural_gas": 500.632*1000}
renewable_indicator = {"solar": 1, "wind": 1, "hydro": 1, "coal": 0, "natural_gas": 0}
consumption_rate = {"solar": 0.90, "wind": 0.85, "hydro": 0.95, "coal": 0.70, "natural_gas": 0.80}

budget = 12 * 10**9  # $9.36 billion
demand = 25263.2 * 1000  # Convert gigawatts to megawatts
renewable_target = 0.5  # Example target, modify if needed

# Define decision variables
x = {s: pulp.LpVariable(f"x_{s}", lowBound=0, upBound=production_limit[s], cat='Continuous') for s in sources}

# Objective function: Minimize total pollution cost
prob += pulp.lpSum(pollution_cost[s] * x[s] for s in sources)

# Constraint: Total energy demand must be met
prob += pulp.lpSum(x[s] for s in sources) >= demand

# Constraint: Budget constraint
prob += pulp.lpSum(lcoe_cost[s] * x[s] for s in sources) <= budget

# Constraint: Renewable energy target
renewable_numerator = pulp.lpSum(x[s] * renewable_indicator[s] for s in sources)
renewable_denominator = pulp.lpSum(x[s] for s in sources)
prob += renewable_numerator >= renewable_target * renewable_denominator

# Constraint: Production limit for each energy source
for s in sources:
    prob += x[s] <= production_limit[s]

# Solve the problem
prob.solve()

# Output results
print("Status:", pulp.LpStatus[prob.status])
for s in sources:
    print(f"{s} Production: {x[s].varValue} MW")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/54aad64fa1a14d4d8ac1bf2a412f1f62-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/54aad64fa1a14d4d8ac1bf2a412f1f62-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 39 RHS
At line 48 BOUNDS
At line 54 ENDATA
Problem MODEL has 8 rows, 5 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-7) rows, 5 (0) columns and 5 (-15) elements
0  Obj 3.6840854e+09 Primal inf 702630.54 (1)
1  Obj 3.7892293e+09
Optimal - objective value 3.7892293e+09
After Postsolve, objective 3.7892293e+09, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 3789229251 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wal

In [41]:
x

{'solar': x_solar,
 'wind': x_wind,
 'hydro': x_hydro,
 'coal': x_coal,
 'natural_gas': x_natural_gas}

In [42]:
print("Total Pollution Cost:", pulp.value(prob.objective))
print("Decision Variable Values:", {s: x[s].varValue for s in sources})

Total Pollution Cost: 3789229251.0
Decision Variable Values: {'solar': 999.0, 'wind': 999.0, 'hydro': 25261202.0, 'coal': 0.0, 'natural_gas': 0.0}


#### The process of finding the best value was not successful, but that does not mean that there is no viable value, just that I don't think it is the best value. Therefore, the next step will still be to try to see the quality of CO2 emissions. Let's continue.

## 4.2 minimize cost

In [43]:
import pulp

# Define the problem
prob = pulp.LpProblem("EnergyMixOptimization", pulp.LpMinimize)

# Given data
sources = ["solar", "wind", "hydro", "coal", "natural_gas"]
lcoe_cost = {"solar": 86.5, "wind": 106.5, "hydro": 213.0, "coal": 74.0, "natural_gas": 54.0}
pollution_cost = {"solar": 37, "wind": 12, "hydro": 150, "coal": 1000, "natural_gas": 430}
fixed_cost = {"solar": 1516000, "wind": 1389000, "hydro": 6428500, "coal": 4074000, "natural_gas": 1300000}
production_limit = {"solar": 905.264, "wind": 905.264, "hydro": 43000.0, "coal": 0.0, "natural_gas": 452.632}
renewable_indicator = {"solar": 1, "wind": 1, "hydro": 1, "coal": 0, "natural_gas": 0}
consumption_rate = {"solar": 0.90, "wind": 0.85, "hydro": 0.95, "coal": 0.70, "natural_gas": 0.80}

budget = 9.36 * 10**9  # $9.36 billion
demand = 45263.2 * 1000  # Convert gigawatts to megawatts
renewable_target = 0.5  # Example target, modify if needed
max_emission = 1.5 * 10**9  # Example maximum emission constraint

# Define decision variables
x = {s: pulp.LpVariable(f"x_{s}", lowBound=0, upBound=production_limit[s], cat='Continuous') for s in sources}

# Objective function: Minimize total cost
prob += pulp.lpSum(lcoe_cost[s] * x[s] for s in sources)

# Constraint: Total energy demand must be met
prob += pulp.lpSum(x[s] for s in sources) >= demand

# Constraint: Budget constraint
prob += pulp.lpSum(lcoe_cost[s] * x[s] for s in sources) <= budget

# Constraint: Emission constraint
prob += pulp.lpSum(pollution_cost[s] * x[s] for s in sources) <= max_emission

# Constraint: Renewable energy target
renewable_numerator = pulp.lpSum(x[s] * renewable_indicator[s] for s in sources)
renewable_denominator = pulp.lpSum(x[s] for s in sources)
prob += renewable_numerator >= renewable_target * renewable_denominator

# Constraint: Production limit for each energy source
for s in sources:
    prob += x[s] <= production_limit[s]

# Solve the problem
prob.solve()

# Output results
print("Status:", pulp.LpStatus[prob.status])
for s in sources:
    print(f"{s} Production: {x[s].varValue} MW (Limit: {production_limit[s]} MW)")
print("Total Emission:", pulp.value(pulp.lpSum(pollution_cost[s] * x[s] for s in sources)))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/01ceda374e774958a9ebf1e2e8c42fa8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/01ceda374e774958a9ebf1e2e8c42fa8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 45 RHS
At line 55 BOUNDS
At line 61 ENDATA
Problem MODEL has 9 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 38128263 (1)
3  Obj 9.6407787e+09 Primal inf 2.374726e+08 (4)
Primal infeasible - objective value 9.6407787e+09
PrimalInfeasible objective 9640778705 - 3 iterations time 0.002

Result - Linear relaxation infeasible

Enumerated nodes:           0
Total iterations:        

## 4.3 Optimizing both cost and environmental impact

In [24]:
# Load the dataset
parameters_df = pd.read_csv("energy_parameters.csv", index_col=0)
parameters_df.iloc[3,1] = 600
parameters_df.iloc[3,3] = 1000
parameters_df.iloc[:,3] *= 1
parameters_df

,LCOE Cost ($/MWh),Pollution Cost (g/kWh CO2eq),Fixed Cost ($ per MW),Production Limit (MW),Renewable Indicator,Consumption Rate
Energy Source,,,,,,
solar,86.5,37,1516000,905.264,1,0.90
wind,106.5,12,1389000,905.264,1,0.85
hydro,213.0,150,6428500,43000.000,1,0.95
coal,74.0,600,4074000,1000.000,0,0.70
natural_gas,54.0,430,1300000,452.632,0,0.80


In [25]:
# Define the parameters
alpha = 0.8  # Weight for cost
beta = 0.2   # Weight for environmental impact
D = 10000    # Total energy demand in MW
lambda_renewable = 0.3  # Minimum percentage of renewable energy

# Create the problem
prob = lp.LpProblem("Energy_Mix_Optimization", lp.LpMinimize)

# Define decision variables
energy_sources = parameters_df.index
x = lp.LpVariable.dicts("EnergyProduction", energy_sources, lowBound=0)

# Objective function: Minimize alpha * cost + beta * pollution
cost = lp.lpSum([parameters_df.loc[i, 'LCOE Cost ($/MWh)'] * x[i] for i in energy_sources])
pollution = lp.lpSum([parameters_df.loc[i, 'Pollution Cost (g/kWh CO2eq)'] * x[i] for i in energy_sources])
prob += alpha * cost + beta * pollution, "Total_Cost_and_Pollution"

# Constraints
# 1. Total energy production must meet demand
prob += lp.lpSum([x[i] for i in energy_sources]) >= D, "Meet_Demand"

# 2. Energy production from each source must not exceed its limit
for i in energy_sources:
    prob += x[i] <= parameters_df.loc[i, 'Production Limit (MW)'], f"Production_Limit_{i}"

# 3. At least lambda_renewable percent of energy must come from renewable sources
renewable_sources = parameters_df[parameters_df['Renewable Indicator'] == 1].index
prob += lp.lpSum([x[i] * parameters_df.loc[i, 'Consumption Rate'] for i in renewable_sources]) >= lambda_renewable * lp.lpSum([x[i] for i in energy_sources]), "Renewable_Energy_Requirement"

# Solve the problem
prob.solve()

# Print the results
print("Status:", lp.LpStatus[prob.status])
print("Total Cost and Pollution:", lp.value(prob.objective))
for v in prob.variables():
    print(v.name, "=", v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8cd2c59a061744eaa08a9c5b5483e85b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/8cd2c59a061744eaa08a9c5b5483e85b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 33 RHS
At line 41 BOUNDS
At line 42 ENDATA
Problem MODEL has 7 rows, 5 columns and 15 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-6) rows, 5 (0) columns and 5 (-10) elements
0  Obj 1350042.6 Primal inf 3263.2605 (1)
1  Obj 1736387.1
Optimal - objective value 1736387.1
After Postsolve, objective 1736387.1, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1736387.139 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds)

## 4.4 model

In [28]:
# Load the provided dataset
parameters_df = pd.read_csv("energy_parameters.csv", index_col=0)

# Modify specific values as per the provided changes
parameters_df.iloc[3, 1] = 600  # Update Pollution Cost for coal
parameters_df.iloc[3, 3] = 1000  # Update Production Limit for coal
parameters_df.iloc[:, 3] *= 1  # Scale Production Limit (no change in this case)

# Display the updated dataframe
print(parameters_df)

               LCOE Cost ($/MWh)  Pollution Cost (g/kWh CO2eq)  \
Energy Source                                                    
solar                       86.5                            37   
wind                       106.5                            12   
hydro                      213.0                           150   
coal                        74.0                           600   
natural_gas                 54.0                           430   

               Fixed Cost ($ per MW)  Production Limit (MW)  \
Energy Source                                                 
solar                        1516000                905.264   
wind                         1389000                905.264   
hydro                        6428500              43000.000   
coal                         4074000               1000.000   
natural_gas                  1300000                452.632   

               Renewable Indicator  Consumption Rate  
Energy Source                           

In [29]:
# Define additional parameters
num_hours = 24  # Number of hours in a day
alpha = 0.5  # Weight for cost
beta = 0.5   # Weight for environmental impact

# Generate synthetic data for missing parts
np.random.seed(42)  # For reproducibility

# Costs and pollution coefficients for variable sources
c_v = 200  # Cost for variable source v
c_w = 100  # Cost for variable source w
p_v = 50   # Pollution for variable source v
p_w = 20   # Pollution for variable source w

# Production limits for variable sources
V_t_max = np.random.uniform(200, 400, num_hours)  # Max limit for v_t
W_t_max = np.random.uniform(100, 300, num_hours)  # Max limit for w_t

# Hourly demand
D_t = np.random.uniform(1000, 2000, num_hours)  # Demand for each hour

# Create the problem
prob = lp.LpProblem("Energy_Mix_Optimization_Time_Varying", lp.LpMinimize)

# Define decision variables
energy_sources = parameters_df.index
hours = range(num_hours)
x = lp.LpVariable.dicts("EnergyProduction", [(i, t) for i in energy_sources for t in hours], lowBound=0)
v = lp.LpVariable.dicts("VariableSourceV", hours, lowBound=0)
w = lp.LpVariable.dicts("VariableSourceW", hours, lowBound=0)

# Objective function
cost = lp.lpSum([parameters_df.loc[i, 'LCOE Cost ($/MWh)'] * x[(i, t)] for i in energy_sources for t in hours]) + lp.lpSum([c_v * v[t] for t in hours]) + lp.lpSum([c_w * w[t] for t in hours])
pollution = lp.lpSum([parameters_df.loc[i, 'Pollution Cost (g/kWh CO2eq)'] * x[(i, t)] for i in energy_sources for t in hours]) + lp.lpSum([p_v * v[t] for t in hours]) + lp.lpSum([p_w * w[t] for t in hours])
prob += alpha * cost + beta * pollution, "Total_Cost_and_Pollution"

# Constraints
for t in hours:
    # Demand constraint
    prob += lp.lpSum([x[(i, t)] for i in energy_sources]) + v[t] + w[t] >= D_t[t], f"Meet_Demand_{t}"
    
    # Production limit constraints for each energy source
    for i in energy_sources:
        prob += x[(i, t)] <= parameters_df.loc[i, 'Production Limit (MW)'], f"Production_Limit_{i}_{t}"
    
    # Variable source limits
    prob += v[t] <= V_t_max[t], f"V_t_Max_{t}"
    prob += w[t] <= W_t_max[t], f"W_t_Max_{t}"

# Solve the problem
prob.solve()

# Print the results
print("Status:", lp.LpStatus[prob.status])
print("Total Cost and Pollution:", lp.value(prob.objective))

# Print the energy production for each source and hour
for t in hours:
    print(f"Hour {t}:")
    for i in energy_sources:
        print(f"  Source {i}: {x[(i, t)].varValue}")
    print(f"  Variable Source V: {v[t].varValue}")
    print(f"  Variable Source W: {w[t].varValue}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bab25a97158a48d491ae20ea21abc6c8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/bab25a97158a48d491ae20ea21abc6c8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 197 COLUMNS
At line 702 RHS
At line 895 BOUNDS
At line 896 ENDATA
Problem MODEL has 192 rows, 168 columns and 336 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 24 (-168) rows, 168 (0) columns and 168 (-168) elements
Perturbing problem by 0.001% of 337 - largest nonzero change 3.5102985e-05 ( 1.720435e-05%) - largest zero change 0
0  Obj 0 Primal inf 36130.083 (24)
24  Obj 2170144.3
Optimal - objective value 2170144
After Postsolve, objective 2170144, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2170144.002 - 24 iterations time 0.002, Pr

## 4.5 Little complex model

In [9]:
alpha,beta = 1,1

x = LpVariable.dicts("x", range(n), lowBound=0)
model = LpProblem("Energy_Optimization", LpMinimize)

model += alpha * lpSum(c[i] * x[i] for i in range(n)) + beta * lpSum(p[i] * x[i] for i in range(n))

#energy supply exceed demand
model += lpSum(x[i] for i in range(n)) >= D

#energy source not exceed limit
for i in range(n):
    model += x[i] <= l[i]

#λ%, propostion of enewable sources
model += lpSum(x[i] * r[i] for i in range(n)) >= lambda_ * lpSum(x[i] for i in range(n))

model.solve()

NameError: name 'n' is not defined

### Optimization for energy storage

In [ ]:
x = LpVariable.dicts("x", [(i, t) for i in sources for t in time_periods], lowBound=0)  
s = LpVariable.dicts("s", time_periods, lowBound=0)  
u = LpVariable.dicts("u", time_periods, lowBound=0)


model += alpha * lpSum(c[i] * x[i, t] for i in sources for t in time_periods) + \
         beta * lpSum(p[i] * x[i, t] for i in sources for t in time_periods)

